In [15]:
!pip install numpy pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 4.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.5/28.5 MB 11.2 MB/s eta 0:00:0000:0100:01


In [16]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
np.random.seed(42)

In [22]:
# df = pd.read_csv("s3://jb-data-set/companies.csv")
df = pd.read_csv("./companies.csv")
df.head(5)

,Company,Industry,Revenue,Employees,#job30,#job90,MonthlyCharge,TotalCharge,Churn
0,comp_0,consulting,36070,9018,3,25,801,2045,1
1,comp_1,retail,94580,23645,9,59,292,1504,1
2,comp_2,service,95310,23828,5,28,316,1053,0
3,comp_3,retail,45580,15193,4,70,170,781,1
4,comp_4,health,25340,6335,9,91,237,849,0


In [23]:
df.describe()

,Revenue,Employees,#job30,#job90,MonthlyCharge,TotalCharge,Churn
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,50048.800000,17809.900000,5.990000,55.450000,637.450000,2461.610000,0.530000
std,29913.861463,11605.686037,2.148502,24.125252,322.963812,2265.728018,0.501614
min,1420.000000,473.000000,3.000000,10.000000,113.000000,156.000000,0.000000
25%,24207.500000,8583.250000,4.000000,36.250000,331.750000,944.500000,0.000000
50%,52790.000000,15965.000000,6.000000,58.000000,613.500000,1563.000000,1.000000
75%,75390.000000,24102.500000,8.000000,74.250000,930.500000,3328.500000,1.000000
max,99850.000000,46625.000000,9.000000,99.000000,1184.000000,11966.000000,1.000000


In [24]:
df["Churn"].value_counts()

1    53
0    47
Name: Churn, dtype: int64

In [26]:
industryCat=df["Industry"].unique()

In [27]:
industry_type = CategoricalDtype(categories=industryCat)

In [28]:
df['Industry'] = df['Industry'].astype(industry_type)
df['Churn'] = df['Churn'].astype('bool')

In [29]:
df=df.drop(['Company'],axis=1)

In [30]:
df = pd.get_dummies(df,columns=['Industry'])

In [31]:
df.head(10)

,Revenue,Employees,#job30,#job90,MonthlyCharge,TotalCharge,Churn,Industry_consulting,Industry_retail,Industry_service,Industry_health,Industry_finance,Industry_gov,Industry_travel,Industry_energy,Industry_IT
0,36070,9018,3,25,801,2045,True,1,0,0,0,0,0,0,0,0
1,94580,23645,9,59,292,1504,True,0,1,0,0,0,0,0,0,0
2,95310,23828,5,28,316,1053,False,0,0,1,0,0,0,0,0,0
3,45580,15193,4,70,170,781,True,0,1,0,0,0,0,0,0,0
4,25340,6335,9,91,237,849,False,0,0,0,1,0,0,0,0,0
5,46860,23430,4,63,1021,1157,True,1,0,0,0,0,0,0,0,0
6,93120,23280,8,49,1163,1889,False,0,0,0,0,1,0,0,0,0
7,30220,7555,7,59,1055,2642,False,0,0,0,0,0,1,0,0,0
8,2480,620,3,20,292,302,True,0,0,0,0,0,0,1,0,0
9,60620,15155,6,85,306,520,False,0,0,0,1,0,0,0,0,0


In [32]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Churn', 1), df['Churn'], test_size = .2, random_state=10)

/var/folders/dp/0b2q97nx3s56mg16tvc2wb140000gr/T/ipykernel_22360/3485171907.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_train, X_test, y_train, y_test = train_test_split(df.drop('Churn', 1), df['Churn'], test_size = .2, random_state=10)


In [33]:
model = RandomForestClassifier(max_depth=5)

In [34]:
X_train.head(5)


,Revenue,Employees,#job30,#job90,MonthlyCharge,TotalCharge,Industry_consulting,Industry_retail,Industry_service,Industry_health,Industry_finance,Industry_gov,Industry_travel,Industry_energy,Industry_IT
90,17900,4475,5,29,179,344,0,0,0,0,0,0,0,1,0
70,32830,16415,4,60,975,1302,0,0,0,0,0,0,1,0,0
50,29340,7335,3,89,1111,3500,0,0,0,0,1,0,0,0,0
44,97130,32377,3,26,344,940,1,0,0,0,0,0,0,0,0
76,66540,33270,7,27,371,465,0,0,0,0,0,0,0,1,0


In [35]:
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=5)

In [36]:
y_predict = model.predict(X_test)

In [37]:
conf_matrix = confusion_matrix(y_test, y_predict)
conf_matrix

array([[9, 2],
       [1, 8]])

In [38]:
tn, fp, fn, tp = conf_matrix.ravel()

In [39]:
sensitivity = tp / (tp + fn)
print('Sensitivity : ', sensitivity )

specificity = tn / (tn + fp)
print('Specificity : ', specificity)

Sensitivity :  0.8888888888888888
Specificity :  0.8181818181818182


In [41]:
prediction=model.predict([[92810,23200,10,100,200,5000,0,0,0,0,0,0,0,1,0]])
prediction

/opt/homebrew/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0.])

In [42]:
prediction=model.predict([[92810,23200,10,100,200,5000,0,0,0,0,1,0,0,0,0]])
prediction

/opt/homebrew/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0.])